# CONVERTING MODEL TO TENSORFLOW LITE

In [23]:
import tensorflow as tf
import numpy as np
from transformers import TFViTModel
import os
import time
import cv2

In [2]:
CONFIGURATION = {
    "CLASS_NAMES" : ['angry', 'happy', 'sad'],
    "BATCH_SIZE" : 32,
    "IMAGE_SIZE" : 224,
    "LEARNING_RATE" : 0.01,
    "N_EPOCHS" : 20,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE" : 0.0,
    "N_FILTERS" : 6,
    "KERNEL_SIZE" : 3,
    "N_STRIDES" : 1,
    "POOL_SIZE" : 2,
    "N_DENSE_1" : 128,
    "N_DENSE_2" : 128,
    "NUM_CLASSES" : 3,
    "PATCH_SIZE" : 16,
}

trainDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/train"
testDirectory = "/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/test"

trainDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='training',
)

valDataset = tf.keras.utils.image_dataset_from_directory(
    trainDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=0.2,
    subset='validation',
)

testDataset = tf.keras.utils.image_dataset_from_directory(
    testDirectory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
    validation_split=None,
    subset=None,
)

trainDataset = trainDataset.prefetch(tf.data.AUTOTUNE)
testDataset = testDataset.prefetch(tf.data.AUTOTUNE)
valDataset = valDataset.prefetch(tf.data.AUTOTUNE)

Found 6799 files belonging to 3 classes.
Using 5440 files for training.


2025-02-03 19:13:37.993556: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-02-03 19:13:37.993603: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-02-03 19:13:37.993607: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1738638817.994063  262425 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1738638817.994352  262425 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 6799 files belonging to 3 classes.
Using 1359 files for validation.
Found 2278 files belonging to 3 classes.


In [3]:
ViTModelKeras = 'Models/EmotionDetectionViT.keras'
kerasModel = tf.keras.models.load_model(ViTModelKeras)
kerasModel.evaluate(testDataset)

2025-02-03 19:14:16.910943: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


72/72 [==============================] - 42s 527ms/step - loss: 0.5523 - accuracy: 0.8749 - top_k_accuracy: 0.9680


[0.5522751212120056, 0.8748902678489685, 0.9679543375968933]

## Converting to Tensorflow Lite

In [74]:
def representativeDatagen():
    for input_value, _ in trainDataset:
        yield [input_value]

In [75]:
converter = tf.lite.TFLiteConverter.from_keras_model(kerasModel)
converter.optimizations = [tf.lite.Optimize.DEFAULT] # Used to quantize the model
converter.inference_input_type = tf.uint8 # Used to convert the model to uint8
converter.inference_output_type = tf.uint8 # Used to convert the model to uint8
converter.representative_dataset = representativeDatagen # Used to calculate scale and zero point for static quantization

In [76]:
tfLiteModelPath = "Models/EmotionDetectionViT.tflite"
tfLiteModel = converter.convert()
with open(tfLiteModelPath, 'wb') as f:
    f.write(tfLiteModel)

INFO:tensorflow:Assets written to: /var/folders/8v/j561wc497p1672n3xvshbbd00000gn/T/tmp5mvgkyfl/assets


INFO:tensorflow:Assets written to: /var/folders/8v/j561wc497p1672n3xvshbbd00000gn/T/tmp5mvgkyfl/assets
/opt/homebrew/Caskroom/miniforge/base/envs/HuggingFaceTransformers/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1738642481.828844  262425 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1738642481.829175  262425 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-02-03 20:14:41.829734: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/8v/j561wc497p1672n3xvshbbd00000gn/T/tmp5mvgkyfl
2025-02-03 20:14:41.842618: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-02-03 20:14:41.842628: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/8v/j561wc497p1672n3xvshbbd00000gn/T/tmp5mvgkyfl
2

In [77]:
def convert_size(size_bytes):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_bytes < 1024:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024
print("TensorFlow Model Size: ", convert_size(os.path.getsize(ViTModelKeras)))
print("TensorFlow Lite Model Size: ", convert_size(os.path.getsize(tfLiteModelPath)))

TensorFlow Model Size:  330.03 MB
TensorFlow Lite Model Size:  83.04 MB


## Tensorflow Lite Runtime

In [78]:
! python3 -m pip install ai-edge-litert

In [79]:
from ai_edge_litert.interpreter import Interpreter

In [80]:
testImage = cv2.imread("/Users/aman/Documents/Work/Machine Learning/Computer-Vision-TensorFlow/Human-Emotions-Detection/Dataset/Emotions Dataset/Emotions Dataset/test/happy/13736.jpg")
testImage = cv2.resize(testImage, (CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"]))
testImage = np.expand_dims(testImage, axis=0)

In [81]:
interpreter = Interpreter(model_path=tfLiteModelPath)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

interpreter.set_tensor(input_details['index'], testImage)
interpreter.invoke()

output_data = interpreter.get_tensor(output_details['index'])[0]
print(CONFIGURATION["CLASS_NAMES"][np.argmax(output_data)])

angry


In [82]:
def evaluate(modelPath, dataset):
    total, acc = 0, 0

    interpreter = Interpreter(model_path=modelPath)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    for images, labels in dataset.take(1):
        for i in range(len(images)):
            image = np.expand_dims(images[i], axis=0).astype(input_details['dtype'])
            interpreter.set_tensor(input_details['index'], image)
            interpreter.invoke()
            output_data = interpreter.get_tensor(output_details['index'])
            if np.argmax(output_data) == np.argmax(labels[i]):
                acc += 1
            total += 1
    return acc / total

In [84]:
accuracy = evaluate(tfLiteModelPath, valDataset)
print("Accuracy TF Lite Model: ", accuracy)

Accuracy TF Lite Model:  0.28550404709345106
